# Nifty 50 Partial Replication 
Index funds have become a household occurence in the financial sphere, especially for retail investors, due to their simplicity, diversification and low cost compared to actively managed funds. Following an index can be done in one of two ways:
- **Full replication:** The most common approach, where a fund holds all constituent stocks present in the index, with the same weights as the index.
- **Partial Replication:** With this approach, only a subset of the index's constituents is selected, which should aim to approximate the overall index performance.

Partial replication can offer a number of benefits over full replication, but mainly:
- Lower transaction costs: Reducing the number of stocks traded minimizes fees such as brokerage fees.
- Reduced Trading fees: By potentially avoiding low-liquidity stocks, we avoid have wider bid-ask spreads, as well as having less market impact.
- Higher flexibility: Funds can add other constraints on top of avoiding low-liquidity stocks, such as those on sectors or specific stocks, giving them better control over their risk management.
Our project's goal is to apply supervised learning techniques to develop a partial replication strategy, minimizing the tracking error of our portfolio.

## Methodology
The project will focus on the NIFTY 100 index, which includes the top 100 companies in the National Stock Exchange of India (NSE) based on market capitalization.
We will begin by gathering the minute-level data for both the index as well as the constituents data. Combining the data into a single dataframe, and perform some basic cleaning tasks, removing certain columns with a large number of empty values, as well as rows without target value, etc. Since our goal is to reduce the number of constituents to use only a subset of the total 100 stocks, our models will follow a 2 step approach:
1. Feature Selection: We will first apply different feature selection techniques, (Lasso regression, Correlation matrix, Mutual information) to identify the most relevant stocks for our model.
2. Model Training: Once the most relevant features (stocks) are selected, we will train different supervised learning models on the reduced dataset, these models will predict the weights for each of the relevant stocks.
To ensure the robustness of the test results, while accounting for the nature of the data (ordered time series) we will use a **sliding window approach**, where:
* A fixed-size window (roughly 1-year) is used to train the model.
* The model will be tested on roughly the next 3 months.
* The window will slide forward, repeating until we use all available data.

For a certain model, the overall out-of-sample error will be the average error over all the windows. The error we will aim to minimize is the **tracking error**, for simplicity models will minimize MSE. Finally, we will compare models by plotting the error vs the number of stocks, to visualize how it evolves as we reduce the size of the subset of constituents chosen.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, ParameterGrid
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore")
warnings.filterwarnings("always", category=ConvergenceWarning)
import seaborn as sns
from sklearn.pipeline import Pipeline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel,mutual_info_regression

## Tracking Error
Tracking error is the metric most commonly used to judge how well a replicating portfolio tracks the benchmark index, for full replication we would expect this to be close to 0 (slightly higher than 0 due to transaction costs or rebalancing delays).

The tracking error is defined as the standard deviation of the differences between the index and replicating portfolio over a specified time period.

$$
\text{Tracking Error} = \sqrt{\frac{\sum_{t=1}^T \left( R_{portfolio}(t) - R_{index}(t) \right)^2}{T}}
$$

Where: 
-  $R_{portfolio}(t)$ is the portfolio return at time $t$,  
-  $R_{index}(t)$ is the index's return at time $t$,  
-  $T$  is the total number of time periods.  

In [1]:
def tracking_error(y_true, y_pred):

    # Convert y_true and y_pred to pandas Series for easy handling of pct_change()
    y_true = pd.Series(y_true)
    y_pred = pd.Series(y_pred)
    
    # Calculate the difference between the percentage changes
    diff_pct_change = y_true - y_pred    

    # Return the standard deviation of the differences (tracking error)
    return np.std(diff_pct_change.dropna())  # dropna() to avoid NaN values in pct_change

In [ ]:
def adj_tracking_error(y_true, X_test, model):
    cost = 0.001  # 0.1% cost
    X_test_adjusted = X_test.copy()
    X_test_adjusted[0, :] -= cost  # Subtract cost from the first return of each stock

    # Predict with adjustment
    y_pred_adjusted = model.predict(X_test_adjusted)
    return tracking_error(y_true, y_pred_adjusted)

# Data

In [ ]:
directory_path = '/kaggle/input/stock-market-data-nifty-50-stocks-1-min-data'
pattern = os.path.join(directory_path, "*.csv")  # Change "*.csv" to match other patterns if needed
Get the list of files that match the pattern
file_list = glob.glob(pattern)

def fetch_data(cols, file_list):
    df_list = []
    for file in tqdm(file_list):
        stock_name = os.path.basename(file).split("_")[0]
        file_df = pd.read_csv(file, index_col=0)
        file_df = file_df[[cols]]
        file_df.columns = pd.MultiIndex.from_product([[stock_name], file_df.columns])
        df_list.append(file_df)
    return pd.concat(df_list, axis=1)

df = fetch_data('close')
df.to_csv('/kaggle/input/nifty_50_closing.csv')

In [ ]:
df = pd.read_csv('/kaggle/input/nifty-50-closing-data/nifty_50_closing.csv', index_col=0, header=[0, 1])
df = df.xs('close', level=1, axis=1)
df.index = pd.to_datetime(df.index)
nan_rows = df.isna().any(axis=1)

# Get the number of rows with NaN value
num_nan_rows = nan_rows.sum()


print(f"Number of rows with NaN values: {num_nan_rows}")

In [ ]:
nifty_100_closing = pd.read_csv('/kaggle/input/nifty-50-minute-data/NIFTY 100_minute.csv', index_col=0)[['close']]
nifty_100_closing.index = pd.to_datetime(nifty_100_closing.index)
nifty_100_closing.rename(columns={'close': 'NIFTY_100'}, inplace=True)

In [ ]:
threshold = 0.10

# Drop columns with more than the threshold percentage of missing values
df = df.dropna(thresh=int((1-threshold) * len(df)), axis=1)

nan_rows = df.isna().any(axis=1)
# Get the number of rows with NaN values
num_nan_rows = nan_rows.sum()
print(f"Number of rows with NaN values: {num_nan_rows}")
df.dropna(axis=0, inplace=True)
df.index = df.index.tz_localize(None)
zero_rows = (df == 0.0).any(axis=1)
num_zero_rows = zero_rows.sum()

print(f"Number of rows with 0.0 values: {num_zero_rows}")

print(f"Total rows {len(df)}")

In [ ]:
df_total = df.join(nifty_100_closing, how='left')

In [ ]:
df_total.dropna(inplace=True)

In [ ]:
# Calculate the percentage change
X_pct_change = df_total.pct_change()

# Define the threshold for large changes (e.g., 100% change)
threshold = 1.0  # 1.0 corresponds to 100% when working with pct_change()

# Find rows with any column having a percentage change greater than the threshold
large_change_rows = X_pct_change[(X_pct_change.abs() > threshold).any(axis=1)]

# Get the number of rows where there are large percentage changes
num_large_changes = large_change_rows.shape[0]

print(f"Number of rows with large changes (>{threshold * 100}%): {num_large_changes}")

In [ ]:
print(len(df_total))

In [ ]:
df_total.sort_index(inplace=True)

# Models

In this section, we will utilize a combined approach to limit the number of features (stocks) and train our models.



1. Feature Selection (using Lasso w/ high alpha, Mutual Info, ANOVA, RFE)

2. Define threshold to drop features

3. Train model on reduced feature space (Hyperparameter tuned)

4. Get tracking error

5. Save pair (number of stocks, tracking error) for plotting

## Correlation - Lasso

In [ ]:
def remove_correlated_features(X_train, threshold):
    X_train = pd.DataFrame(X_train)
    
    # Compute the correlation matrix (absolute values)
    corr_matrix = X_train.corr().abs()
    
    # Create a mask to select only the upper triangle of the correlation matrix
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than the threshold
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    print(f"Features to drop (correlation > {threshold}): {to_drop}")
    selected_features = [i for i in range(X_train.shape[1]) if i not in to_drop]
    
    return selected_features
    

In [ ]:
from sklearn.preprocessing import StandardScaler


def Corr_Lasso(threshold, X, y, debug=False):
    tscv = TimeSeriesSplit(n_splits=18, max_train_size=85000, test_size=30000)
    stocks = []
    num_stocks = []
    fold_scores = []
    adj_fold_scores = []

    for train_index, test_index in tscv.split(X):
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train_cv)

        # Feature selection using correlation matrix
        selected_indices = remove_correlated_features(X_scaled, threshold)

        X_train_selected = X_scaled[:, selected_indices]
        X_test_selected = X_test_cv.to_numpy()[:, selected_indices]
        if debug == True:
            print(f"Selected {len(selected_indices)} stocks")
        num_stocks.append(len(selected_indices))
        stocks.append(selected_indices)

        best_alpha = None
        best_score = float('inf')  # Assuming lower is better for your metric
        for alph in [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.002, 0.0025]:
            lasso = Lasso(alpha=alph, max_iter=15000)
            lasso.fit(X_train_selected, y_train_cv)
            y_pred = lasso.predict(X_test_selected)
            fold_score = tracking_error(y_test_cv.values, y_pred)
            adj_fold_score = adj_tracking_error(y_test_cv.values, X_test_selected, lasso)
            
            if fold_score < best_score:
                best_score = fold_score
                best_alpha = alph

        print(f"Best Alpha for this fold: {best_alpha}, Best Fold Score: {best_score}")
        fold_scores.append(best_score)
        adj_fold_scores.append(adj_fold_score)

    # Calculate the average score across all folds
    average_score = np.mean(fold_scores)
    average_adj_score = np.mean(adj_fold_scores)

    # Get average stocks chosen over all folds
    average_selected_features = np.mean(num_stocks)

    # Print the tracking error for this alpha
    print(f"Threshold: {threshold}, Tracking Error: {average_score}, Adj Tracking Error: {average_adj_score}")
    return stocks, average_selected_features, average_score, average_adj_score

In [ ]:
def Corr_Lasso_plot(X,y):
    thresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    for threshold in thresholds:
        print(f"----------Starting threshold: {threshold}-------------")
        _, num_stocks, tracking_err, adj_tracking_err = Corr_Lasso(threshold, X, y, debug=False)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    color = 'tab:blue'    
    ax1.set_xlabel('Alpha')
    ax1.set_ylabel('Number of Stocks', color=color)    
    ax1.plot(thresholds, num_stocks_list, color=color, marker='o', label='Number of Stocks')
    ax1.tick_params(axis='y', labelcolor=color)
    
    # Creating a second y-axis for tracking error
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Tracking Error', color=color)
    #ax2.plot(alphas, tracking_errors, color='tab:green', marker='o', linestyle='--', label='Tracking Error')
    ax2.plot(thresholds, adj_tracking_errors, color=color, marker='x', linestyle='--', label='Adjusted Tracking Error')
    ax2.tick_params(axis='y', labelcolor=color)
    
    # Adding a title and legend
    plt.title('Number of Stocks and Tracking Error vs. Alpha')    
    fig.tight_layout()  # Adjust layout to prevent overlap
    plt.savefig('Corr-Lasso.png')
    plt.show()

## Lasso - Lasso



We will utilize a Lasso model with a high alpha value to perform feature selection, this will drop the coefficients for less important features to 0. After that we retrain a Lasso model with hyperparameter tuning.

In [ ]:
from sklearn.preprocessing import StandardScaler



def Lasso_Lasso(alpha, X, y):    
    tscv = TimeSeriesSplit(n_splits=18, max_train_size=85000, test_size=30000)
    stocks = []
    num_stocks = []
    fold_scores = []
    adj_fold_scores = []

    for train_index, test_index in tscv.split(X):
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]
        train_period = X.iloc[train_index].index
        test_period = X.iloc[test_index].index

        # Convert to date format
        train_start_date, train_end_date = train_period.min(), train_period.max()
        test_start_date, test_end_date = test_period.min(), test_period.max()

        print(f"Train period: {train_start_date} to {train_end_date}")
        print(f"Test period: {test_start_date} to {test_end_date}")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train_cv)

        # Feature selection with alpha from list
        lasso_fs = Lasso(alpha=alpha, max_iter=200)
        lasso_fs.fit(X_scaled, y_train_cv)

        #for feature, coef in zip(X_train_cv.columns, lasso_fs.coef_)
        #    print(f"{feature}: {coef}")

        selector = SelectFromModel(lasso_fs, threshold=10e-10, max_features=None)  # Adjust threshold as needed
        X_selected_train = selector.transform(X_scaled)
        X_selected_test = selector.transform(X_test_cv)

        selected_features = np.where(selector.get_support())[0] 
        print(f"Selected {len(selected_features)} stocks")
        num_stocks.append(len(selected_features))
        stocks.append(selected_features)

        best_alpha = None
        best_score = float('inf')  # Assuming lower is better for your metric
        for alph in [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.002, 0.0025]:
            lasso = Lasso(alpha=alph, max_iter=15000)
            lasso.fit(X_selected_train, y_train_cv)
            y_pred = lasso.predict(X_selected_test)
            fold_score = tracking_error(y_test_cv.values, y_pred)
            adj_fold_score = adj_tracking_error(y_test_cv.values, X_selected_test, lasso)
            # print(f"Alpha: {alpha}, Fold Score: {fold_score}"
            
            if fold_score < best_score:
                best_score = fold_score
                best_alpha = alph

        print(f"Best Alpha for this fold: {best_alpha}, Best Fold Score: {best_score}")
        fold_scores.append(best_score)
        adj_fold_scores.append(adj_fold_score)

    # Calculate the average score across all folds
    average_score = np.mean(fold_scores)
    average_adj_score = np.mean(adj_fold_scores)

    # Get average stocks chosen over all folds
    average_selected_features = np.mean(num_stocks)

    # Print the tracking error for this alpha
    print(f"Alpha: {alpha}, Average Tracking Error: {average_score}")
    return stocks, average_selected_features, average_score, average_adj_score

In [ ]:
def Lasso_Lasso_plot(X,y):
    alphas = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009]
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    for alpha in alphas:
        print(f"----------Starting alpha {alpha}-------------")
        _, num_stocks, tracking_err, adj_tracking_err = Lasso_Lasso(alpha, X, y)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    color = 'tab:blue'    
    ax1.set_xlabel('Alpha')
    ax1.set_ylabel('Number of Stocks', color=color)    
    ax1.plot(alphas, num_stocks_list, color=color, marker='o', label='Number of Stocks')
    ax1.tick_params(axis='y', labelcolor=color)
    
    # Creating a second y-axis for tracking error
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Tracking Error', color=color)
    #ax2.plot(alphas, tracking_errors, color='tab:green', marker='o', linestyle='--', label='Tracking Error')
    ax2.plot(alphas, adj_tracking_errors, color=color, marker='x', linestyle='--', label='Adjusted Tracking Error')
    ax2.tick_params(axis='y', labelcolor=color)
    
    # Adding a title and legend
    plt.title('Number of Stocks and Tracking Error vs. Alpha')    
    fig.tight_layout()  # Adjust layout to prevent overlap
    plt.savefig('Lasso-Lasso.png')
    plt.show()

### Mutual Info - Lasso

Using mutual information between variables, which measures the dependency between them, 0 value being independence. By getting the mutual information between the features (stocks) and the target (NIFTY 100) we can "discard" the variables with lower dependencies. After feature selection, we traing a Lasso model with CV.

In [ ]:
from sklearn.preprocessing import StandardScaler


def MI_Experiment(percentile, X, y, model_class, params, debug=False):
    tscv = TimeSeriesSplit(n_splits=18, max_train_size=85000, test_size=30000)
    stocks = []
    num_stocks = []
    fold_scores = []
    adj_fold_scores = []

    for train_index, test_index in tscv.split(X):
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train_cv)

        # Feature selection with alpha from list
        mi_scores = mutual_info_regression(X_train_cv, y_train_cv)
        mi_scores_series = pd.Series(mi_scores, index=X_train_cv.columns)

        threshold_value = mi_scores_series.quantile(percentile)        

        selected_features = mi_scores_series[mi_scores_series > threshold_value].index
        selected_indices = [X_train_cv.columns.get_loc(feature) for feature in selected_features]

        # Filter the NumPy arrays based on the selected feature indices
        X_train_selected = X_scaled[:, selected_indices]
        X_test_selected = X_test_cv.to_numpy()[:, selected_indices]

        if debug == True:
            print(f"Selected {len(selected_features)} stocks")
        num_stocks.append(len(selected_features))
        stocks.append(selected_features)

        best_params = None
        best_score = float('inf')  # Assuming lower is better for your metric
        best_adj_score = float('inf')
        for param in ParameterGrid(params):
            if debug == True:
                print(f"Trying combination {param}")
            model = model_class(**param)
            model.fit(X_train_selected, y_train_cv)

            y_pred = model.predict(X_test_selected)
            fold_score = tracking_error(y_test_cv.values, y_pred)
            adj_fold_score = adj_tracking_error(y_test_cv.values, X_test_selected, model)

            if adj_fold_score < best_adj_score:
                best_score = fold_score
                best_adj_score = adj_fold_score
                best_params = param
        if debug == True:
            print(f"Best Params for this fold: {best_params}, Best Fold Score: {best_score}")
        fold_scores.append(best_score)
        adj_fold_scores.append(best_adj_score)

    # Calculate the average score across all folds
    average_score = np.mean(fold_scores)
    average_adj_score = np.mean(adj_fold_scores)

    # Get average stocks chosen over all folds
    average_selected_features = np.mean(num_stocks)

    # Print the tracking error for this alpha
    print(f"Percentile: {percentile}, Tracking Error: {average_score}, Adj Tracking Error: {average_adj_score}")
    return stocks, average_selected_features, average_score, average_adj_score

### MI - Lasso

In [ ]:
def MI_Lasso_plot(X,y):
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    percentiles = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    param_grid = {
        'alpha': [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.002, 0.0025],
        'max_iter': [15000]
    }
    
    for percentile in percentiles:
        print(f"Starting percentile: {percentile}")
        _, num_stocks, tracking_err, adj_tracking_err = MI_Experiment(percentile, X, y, Lasso, param_grid)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    plt.plot(num_stocks_list, adj_tracking_errors, marker='x')
    plt.xlabel('Number of Stocks')
    plt.ylabel('Tracking Error')
    plt.title('Lasso - Tracking Errors vs Number of Stocks')
    plt.grid(True)
    plt.savefig("MI-Lasso.png")
    plt.show()

### MI - ElasticNet

In [ ]:
def MI_ElasticNet_plot(X,y):
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    percentiles = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    param_grid = {
        'alpha': [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.002, 0.0025],
        'l1_ratio': [0.1, 0.5, 0.7, 0.9]
    }
    
    for percentile in percentiles:
        print(f"Starting percentile: {percentile}")
        _, num_stocks, tracking_err, adj_tracking_err = MI_Experiment(percentile, X, y, ElasticNet, param_grid)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    plt.plot(num_stocks_list, adj_tracking_errors, marker='x')
    plt.xlabel('Number of Stocks')
    plt.ylabel('Tracking Error')
    plt.title('ElasticNet - Tracking Errors vs Number of Stocks ')
    plt.grid(True)
    plt.savefig("MI-ElasticNet.png")
    plt.show()

TODO: Theil-Sein Linear Regression

### MI-RandomForestRegressor

In [ ]:
def MI_RF_plot(X,y):
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    percentiles = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    param_grid = {
        'n_estimators': [10, 20, 30, 40, 50],       # Number of trees in the forest
        'max_depth': [5, 10, 15],      # Maximum depth of the tree
        'max_samples': [0.5],
        'min_samples_split': [0.05],
        'bootstrap': [True],
        'n_jobs': [-1],
        'max_features': ['sqrt']
    }
    
    for percentile in percentiles:
        print(f"Starting percentile: {percentile}")
        _, num_stocks, tracking_err, adj_tracking_err = MI_Experiment(percentile, X, y, RandomForestRegressor, param_grid)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    plt.plot(num_stocks_list, adj_tracking_errors, marker='x')
    plt.xlabel('Number of Stocks')
    plt.ylabel('Tracking Error')
    plt.title('RandomForestRegressor - Tracking Errors vs Number of Stocks ')
    plt.grid(True)
    plt.savefig("MI-RandomForestRegressor.png")
    plt.show()

### MI - XGBoost

In [ ]:
def MI_XGBoost_plot(X,y):
    tracking_errors = []
    adj_tracking_errors = []
    num_stocks_list = []
    
    percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    param_grid = {
        'n_estimators': [50, 100, 200, 250],  # Number of boosting rounds
        'learning_rate': [0.01, 0.1, 0.2, 0.25],  # Step size shrinkage
        'max_depth': [3, 5, 7, 10],  # Maximum depth of a tree
        'n_jobs': [-1],
        'tree_method': ['gpu_hist'],
    }
    
    for percentile in percentiles:
        print(f"Starting percentile: {percentile}")
        _, num_stocks, tracking_err, adj_tracking_err = MI_Experiment(percentile, X, y, xgb.XGBRegressor, param_grid)
        num_stocks_list.append(num_stocks)
        tracking_errors.append(tracking_err)
        adj_tracking_errors.append(adj_tracking_err)
    
    # Plotting the results
    fig, ax1 = plt.subplots()
    
    # Plotting number of stocks on the primary y-axis
    plt.plot(num_stocks_list, adj_tracking_errors, marker='x')
    plt.xlabel('Number of Stocks')
    plt.ylabel('Tracking Error')
    plt.title('XGBRegressor - Tracking Errors vs Number of Stocks ')
    plt.grid(True)
    plt.savefig("MI-XGBRegressor.png")
    plt.show()

In [ ]:
df_total_z = df_total.replace(0, np.nan)  # Convert zeroes back to NaNs
df_total_returns = df_total_z.pct_change().dropna()
df_total_returns = 100*df_total_returns
X = df_total_returns.drop(columns=['NIFTY 50', 'NIFTY BANK', 'NIFTY_100'])
y = df_total_returns['NIFTY_100']

In [ ]:
Corr_Lasso_plot(X,y)

In [ ]:
Lasso_Lasso_plot(X,y)

In [ ]:
MI_Lasso_plot(X,y)